## Logistic Regression - COVID-19

### Group Members
- Ramya Prakash
- Soumyadip Mitra
- Kirk Mason (James Mason)
- Hari Chamlagai
- Phil Nguyen

### Introductory Paragraph:

In [1]:
## Implement the model discussed in the post.

#### Question 1: If you were asked to perform the analysis on other states such as North Carlolina, can you obtain the data?

##### Answer:

#### Question 2: Are there any problems with state level data or is it all reported in exactly the same way?

##### Answer: 